In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pokemon/report_Pokemon.pdf
/kaggle/input/pokemon/pokemon_alopez247.csv


In [2]:
!pip install tensorflow

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.1 MB/s eta 0:00:00
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77058 sha256=41092aab95f29b814b9f1b78b2073678a0fbf36cb308430833ec40d2e84766a4
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.14.0
    Uninstalling wrapt-1.14.0:
      Successfully uninstalled wrapt-1.14.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attemptin

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

###Dataset used: https://www.kaggle.com/datasets/alopez247/pokemon

In [4]:
df = pd.read_csv("../input/pokemon/pokemon_alopez247.csv")
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

###Selecting only the required data here

In [5]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

In [6]:
df['isLegendary']

0      False
1      False
2      False
3      False
4      False
       ...  
716     True
717     True
718     True
719     True
720     True
Name: isLegendary, Length: 721, dtype: bool

###isLegendary column contains True/False data so converting to numerical data as 0's and 1's

In [7]:
df['isLegendary'] = df['isLegendary'].astype(int)
df['isLegendary']

0      0
1      0
2      0
3      0
4      0
      ..
716    1
717    1
718    1
719    1
720    1
Name: isLegendary, Length: 721, dtype: int64

###Looking at type1 cloumn:

In [8]:
df['Type_1']

0        Grass
1        Grass
2        Grass
3         Fire
4         Fire
        ...   
716       Dark
717     Dragon
718       Rock
719    Psychic
720       Fire
Name: Type_1, Length: 721, dtype: object

###The values are grass, fire etc so ceating dummy variables here

In [9]:
def dummy_creation(df, dummy_cat):
    for i in dummy_cat:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df, df_dummy], axis = 1)
        df = df.drop(i, axis = 1)
    return(df)

###Using the above function to create dummies for multiple columns

In [10]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])
df.head

<bound method NDFrame.head of      isLegendary  Generation   HP  Attack  Defense  Sp_Atk  Sp_Def  Speed  \
0              0           1   45      49       49      65      65     45   
1              0           1   60      62       63      80      80     60   
2              0           1   80      82       83     100     100     80   
3              0           1   39      52       43      60      50     65   
4              0           1   58      64       58      80      65     80   
..           ...         ...  ...     ...      ...     ...     ...    ...   
716            1           6  126     131       95     131      98     99   
717            1           6  108     100      121      81      95     95   
718            1           6   50     100      150     100     150     50   
719            1           6   80     110       60     150     130     70   
720            1           6   80     110      120     130      90     70   

     Height_m  Weight_kg  ...  Ghost  Grass  

###Now splitting the data into train and test

In [11]:
df['Generation']

0      1
1      1
2      1
3      1
4      1
      ..
716    6
717    6
718    6
719    6
720    6
Name: Generation, Length: 721, dtype: int64

In [12]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]
    
    df_train = df_train.drop(column, axis = 1)
    df_test = df_test.drop(column, axis = 1)
    
    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

###Removing the isLegendary Category from dataframe since it is what the model should predict

In [13]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis = 1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label, axis = 1).values
    test_labels = df_test[label].values
    
    return(train_data, test_data, train_labels, test_labels)

train_data, test_data, train_labels, test_labels = label_delineator(df_train, df_test, 'isLegendary')

###Normalizing the data

In [14]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

###Creating the neural network

Using keras to create the model with 500 neurons, relu activation and activation layer with 2 classes.

In [15]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

2022-04-27 03:10:37.713238: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


###Compiling the model with SGD optimization and sparse categorical crossentropy

In [16]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

###Fit the data on Training Set

In [17]:
model.fit(train_data, train_labels,epochs = 400)

2022-04-27 03:10:38.364217: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/400
18/18 [==============================] - 1s 2ms/step - loss: 0.6369 - accuracy: 0.7123
Epoch 2/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4796 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3973 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3511 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3228 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3041 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2908 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2810 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2731 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2667 - accuracy: 0.9263
Epoch 11/

In [18]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0344 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


###Now to predict and check the predictions

In [19]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'Correctly predicted as:\"{test_labels[index]}\"')
    else:
        print(f'Wrongly Predicted as: \"{prediction[index]}\" it was \"{test+labels[index]}\"')
        return(prediction)

In [20]:
df1 = pd.read_csv("../input/pokemon/pokemon_alopez247.csv")

In [21]:
index = 149 
print(df1.loc[index, 'Name'])
print(df.loc[index, 'isLegendary'])
predictor(test_data, test_labels, index)

Mewtwo
1.0
Correctly predicted as:"1"
